In [1]:
#!pip install torchcontrib

In [2]:
# Torch, Sklearn imports
import torch
from sklearn.model_selection import train_test_split
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from torchcontrib.optim import SWA

## Embeddings
import allennlp
from allennlp.modules.elmo import Elmo, batch_to_ids
from gensim.models import KeyedVectors

import warnings
warnings.filterwarnings("ignore")

In [3]:
import pandas as pd
import numpy as np
import json, re

# Torch, Sklearn imports
from sklearn.model_selection import train_test_split
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
print(torch.__version__)

1.2.0


In [4]:
## NLP libs
from nltk import download
import gensim
from nltk.corpus import stopwords
download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rsilvei/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Load Dataset

In [5]:
dataset = pd.DataFrame(columns = ['phrase', 'intent'])
for intent in ['AddToPlaylist', 'BookRestaurant', 'GetWeather', 'PlayMusic', 'RateBook', 'SearchCreativeWork',
               'SearchScreeningEvent']:
    with open("./2017-06-custom-intent-engines/" + intent + "/train_" + intent + ".json",
              encoding='cp1251') as data_file:
        data = json.load(data_file)
    print("Intent: {}, Length: {}".format(intent,len(data[intent])))
    texts = []
    for i in range(len(data[intent])):
        text = ''
        for j in range(len(data[intent][i]['data'])):
            text += data[intent][i]['data'][j]['text']
        dataset = dataset.append({'phrase': text, 'intent': intent}, ignore_index=True)

Intent: AddToPlaylist, Length: 300
Intent: BookRestaurant, Length: 300
Intent: GetWeather, Length: 300
Intent: PlayMusic, Length: 300
Intent: RateBook, Length: 300
Intent: SearchCreativeWork, Length: 300
Intent: SearchScreeningEvent, Length: 300


In [6]:
def transformText(text, do_stop=False, do_stem=False):
    stops = set(stopwords.words("english"))
    # Convert text to lower
    text = text.lower()
    
    # Cleaning input
    text = text.replace("'s","")
    text = text.replace("’s","")
    text = text.replace("?","")
    text = text.replace("-","")
    
    # Removing non ASCII chars    
    text = re.sub(r'[^\x00-\x7f]',r' ',text)
    # Strip multiple whitespaces
    text = gensim.corpora.textcorpus.strip_multiple_whitespaces(text)
    # Removing all the stopwords
    if (do_stop==True):
        filtered_words = [word for word in text.split() if word not in stops]
    else:
        filtered_words = [word for word in text.split()]
    # Preprocessed text after stop words removal
    text = " ".join(filtered_words)
    # Remove the punctuation
    text = gensim.parsing.preprocessing.strip_punctuation2(text)
    # Strip multiple whitespaces
    text = gensim.corpora.textcorpus.strip_multiple_whitespaces(text)
    if (do_stem==True):
        # Stemming
        text = gensim.parsing.preprocessing.stem_text(text)
    return text

def strip_punctuation(s):
    return ''.join(c for c in s if c not in PUNCT)

## Lemmatization function based on Spacy Library
def lemmatizer_spacy(text):        
    sent = []
    doc = spacy_en(text)
    for word in doc:
        if word.lemma_ == "-PRON-":
            sent.append(word.text)
        else:
            sent.append(word.lemma_)
    return " ".join(sent)

def strip_punctuation(s):
    return ''.join(c for c in s if c not in punctuation)

In [7]:
dataset['preproc_text'] = dataset['phrase'].apply(lambda x: transformText(x))

In [8]:
## Build label vocabulary
label_to_ix = {}
for label in dataset.intent:
    for word in label.split():
        if word not in label_to_ix:
            label_to_ix[word]=len(label_to_ix)

In [9]:
class Intents(Dataset):
    def __init__(self, dataframe, w2v_weights_path):
        self.len = len(dataframe)
        self.label_to_ix = {}
        self.data = dataframe
        self.w2v = w2v = KeyedVectors.load_word2vec_format(w2v_weights_path, binary = True)
        
    def __getitem__(self, index):
        phrase = self.data.preproc_text[index]
        X, _  = self.get_avg_sentence_vector(phrase)
        y = label_to_ix[self.data.intent[index]]
        return X, y
    
    def __len__(self):
        return self.len

    def get_avg_sentence_vector(self, sentence):
        featureVec = np.zeros((self.w2v.vector_size), dtype="float32")
        nwords = 0
        not_found_words = []
        for word in sentence.split():
            if word in self.w2v.index2word:
                nwords = nwords+1
                featureVec = np.add(featureVec, self.w2v.get_vector(word))
            else:
                not_found_words.append(word)
        if nwords>0:
            featureVec = np.divide(featureVec, nwords)
        return featureVec, not_found_words

In [10]:
# Set data locations for embeddings
elmo_config_key_path = '../../../vectors/elmo_2x4096_512_2048cnn_2xhighway_5.5B_options.json'
elmo_weights_key_path = '../../../vectors/elmo_2x4096_512_2048cnn_2xhighway_5.5B_weights.hdf5'
w2v_weights_path = '../../../vectors/GoogleNews-vectors-negative300.bin'

In [11]:
!ls ../../../vectors/

GoogleNews-vectors-negative300.bin
crawl-300d-2M.vec
elmo_2x4096_512_2048cnn_2xhighway_5.5B_options.json
elmo_2x4096_512_2048cnn_2xhighway_5.5B_weights.hdf5
glove.42B.300d.txt
glove.840B.300d.txt
lid.176.ftz
program
uncased_L-12_H-768_A-12
uncased_L-24_H-1024_A-16
wiki-news-300d-1M-subword.txt
wiki-news-300d-1M.txt


In [12]:
train_size = 0.8
train_dataset=dataset.sample(frac=train_size,random_state=200).reset_index(drop=True)
test_dataset=dataset.drop(train_dataset.index).reset_index(drop=True)
print("FULL Dataset: {}".format(dataset.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

FULL Dataset: (2100, 3)
TRAIN Dataset: (1680, 3)
TEST Dataset: (420, 3)


In [13]:
training_set = Intents(train_dataset,  w2v_weights_path)
testing_set = Intents(test_dataset, w2v_weights_path)

## Simple MLP Classifier

In [14]:
class SimpleMLP(nn.Module):
    def __init__(self, inputdim, 
                        nclasses, 
                        nhidden, 
                        dropout = 0,
                        cudaEfficient=True):
        super(SimpleMLP, self).__init__()
        """
        PARAMETERS:
        -dropout:    dropout for MLP
        """
        
        self.inputdim = inputdim
        self.hidden_dim = nhidden
        self.dropout = dropout
        self.nclasses = nclasses
        
        if cudaEfficient:
            self.model = nn.Sequential(
                nn.Linear(self.inputdim, nhidden),
                nn.Dropout(p=self.dropout),
                nn.ReLU(),
                nn.Linear(nhidden, self.nclasses),
                ).cuda()
        else:
            self.model = nn.Sequential(
                nn.Linear(self.inputdim, nhidden),
                nn.Dropout(p=self.dropout),
                nn.ReLU(),
                nn.Linear(nhidden, self.nclasses),
                )
    def forward(self, x):
        log_probs = self.model(x)
        return log_probs

In [15]:
INP_DIM = training_set.w2v.vector_size
NUM_LABELS = len(label_to_ix)
NHIDDEN = 512
DROPOUT = 0.2
model = SimpleMLP(inputdim = INP_DIM ,
          nhidden = NHIDDEN,
          nclasses = NUM_LABELS,
          dropout = DROPOUT, 
          cudaEfficient = False)

In [16]:
# # Parameters
# params = {'batch_size': 64,
#           'shuffle': True,
#           'num_workers': 1}
# training_loader = DataLoader(training_set, **params)
# testing_loader = DataLoader(testing_set, **params)
# loss_function = nn.CrossEntropyLoss()
# learning_rate = 0.001
# optimizer = optim.Adam(params =  model.parameters(), lr=learning_rate)

In [17]:
# max_epochs = 5
# for epoch in range(max_epochs):
#     print("EPOCH -- {}".format(epoch))
#     for i, (sent, label) in enumerate(training_loader):
#         optimizer.zero_grad()
#         sent = Variable(sent)
#         label = Variable(label)
#         output = model.forward(sent)
#         _, predicted = torch.max(output.data, 1)
#         loss = loss_function(output, label)
#         loss.backward()
#         optimizer.step()
#         if i%100 == 0:
#             correct = 0
#             total = 0
#             for sent, label in testing_loader:
#                 sent = Variable(sent)
#                 label = Variable(label)
#                 output = model.forward(sent)
#                 _, predicted = torch.max(output.data, 1)
#                 total += label.size(0)
#                 correct += (predicted.cpu() == label.cpu()).sum()
#             accuracy = 100.00 * correct.numpy() / total
#             # Print Loss
#             print('Iteration: {}. Loss: {}. Accuracy: {}%'.format(i, loss.item(), accuracy))

In [33]:
def get_reply(phrase, model):
    inp, _ = training_set.get_avg_sentence_vector(phrase)
    inp = Variable(torch.Tensor(inp))
    output = model.forward(inp)
    _, predicted = torch.max(output.data, 0)
    pred_label=list(label_to_ix.keys())[list(label_to_ix.values()).index(predicted.item())]
    return pred_label

In [19]:
get_reply("i need to book a restaurant today")

'RateBook'

In [20]:
get_reply("play music")

'SearchScreeningEvent'

In [21]:
get_reply("Obama is cool")

'SearchScreeningEvent'

## Using Stochastic Weight Averaging (SWA) optimizer

In [22]:
# Parameters
params = {'batch_size': 64,
          'shuffle': True,
          'num_workers': 1}
training_loader = DataLoader(training_set, **params)
testing_loader = DataLoader(testing_set, **params)
loss_function = nn.CrossEntropyLoss()
learning_rate = 0.001
base_opt = optim.Adam(params =  model.parameters(), lr=learning_rate)
opt = SWA(base_opt, swa_start=10, swa_freq=5, swa_lr=0.05)

In [23]:
max_epochs = 3
for epoch in range(max_epochs):
    print("EPOCH -- {}".format(epoch))
    for i, (sent, label) in enumerate(training_loader):
        opt.zero_grad()
        sent = Variable(sent)
        label = Variable(label)
        output = model.forward(sent)
        _, predicted = torch.max(output.data, 1)
        loss = loss_function(output, label)
        loss.backward()
        opt.step()
        if i%100 == 0:
            correct = 0
            total = 0
            for sent, label in testing_loader:
                sent = Variable(sent)
                label = Variable(label)
                output = model.forward(sent)
                _, predicted = torch.max(output.data, 1)
                total += label.size(0)
                correct += (predicted.cpu() == label.cpu()).sum()
            accuracy = 100.00 * correct.numpy() / total
            # Print Loss
            print('Iteration: {}. Loss: {}. Accuracy: {}%'.format(i, loss.item(), accuracy))
opt.swap_swa_sgd()

EPOCH -- 0
Iteration: 0. Loss: 1.952578067779541. Accuracy: 51.666666666666664%
EPOCH -- 1
Iteration: 0. Loss: 0.18529416620731354. Accuracy: 92.85714285714286%
EPOCH -- 2
Iteration: 0. Loss: 0.09568418562412262. Accuracy: 97.61904761904762%


In [24]:
torch.save(model.state_dict(), 'model_elmo_swag_uncertainty.pth')

In [25]:
get_reply("i need to book a restaurant today")

'BookRestaurant'

In [26]:
get_reply("play music")

'PlayMusic'

In [27]:
get_reply("Obama is cool")

'GetWeather'

In [30]:
!ls -lah

total 987120
drwxr-xr-x  10 rsilvei  ADESI\Domain Users   320B Aug 30 10:01 .
drwxr-xr-x  24 rsilvei  ADESI\Domain Users   768B Aug 21 10:43 ..
drwxr-xr-x   4 rsilvei  ADESI\Domain Users   128B Aug 26 10:41 .ipynb_checkpoints
-rw-r--r--   1 rsilvei  ADESI\Domain Users    30K Aug 19 15:14 1.text_classifier_roberta.ipynb
-rw-r--r--   1 rsilvei  ADESI\Domain Users    20K Aug 30 10:01 2.uncertainty_swag.ipynb
drwxr-xr-x@ 10 rsilvei  ADESI\Domain Users   320B Aug 19 09:36 2017-06-custom-intent-engines
-rw-r--r--   1 rsilvei  ADESI\Domain Users   617K Aug 30 09:58 model_elmo_swag_uncertainty.pth
-rw-r--r--@  1 rsilvei  ADESI\Domain Users   446K Aug 19 15:10 roberta-base-merges.txt
-rw-r--r--@  1 rsilvei  ADESI\Domain Users   478M Aug 19 15:42 roberta-base-pytorch_model.bin
-rw-r--r--@  1 rsilvei  ADESI\Domain Users   878K Aug 19 15:13 roberta-base-vocab.json


## Load model

In [38]:
modelito = SimpleMLP(inputdim = INP_DIM ,
          nhidden = NHIDDEN,
          nclasses = NUM_LABELS,
          dropout = DROPOUT, 
          cudaEfficient = False)

In [32]:
modelito.load_state_dict(torch.load('model_elmo_swag_uncertainty.pth', map_location='cpu'))

<All keys matched successfully>

In [34]:
get_reply("Obama is cool", modelito)

'GetWeather'

In [35]:
get_reply("play music", modelito)

'PlayMusic'

In [36]:
get_reply("i need to book a restaurant today", modelito)

'BookRestaurant'

In [37]:
import swag

ModuleNotFoundError: No module named 'swag'